
# 🛡️ AI Transaction Guardian – Portfolio Notebook

This notebook demonstrates an end-to-end **transaction risk scoring / fraud detection** workflow suitable for a product portfolio project.

It includes:

1. Data loading (with a fallback synthetic dataset if no CSV is available)
2. Exploratory data analysis (EDA)
3. Feature engineering & preprocessing
4. Model training (Logistic Regression + Random Forest)
5. Evaluation (ROC-AUC, precision/recall, confusion matrix)
6. Threshold tuning
7. A reusable **`score_transactions`** function to generate risk scores for new data

> 🔧 **TODO (when you wire this to real data):**  
> - Point `DATA_PATH` at your real transaction CSV  
> - Align column names & data types with your production schema


In [ ]:

# Core imports
import os
import numpy as np
import pandas as pd

# Modeling & evaluation
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix,
    precision_recall_curve
)

import joblib


## 1. Load data

In [ ]:

# Path to your real transaction CSV
# Example expected columns (customize as needed):
# - transaction_id
# - customer_id
# - amount
# - merchant_category
# - channel (e.g., 'ecom', 'pos', 'ivr')
# - country
# - is_fraud (0/1 label)
DATA_PATH = "data/transactions.csv"  # <-- Update this to your real path

def load_or_generate_data(path: str, n_rows: int = 5000, random_state: int = 42) -> pd.DataFrame:
    """Load a CSV if it exists, otherwise generate a synthetic dataset."""
    if os.path.exists(path):
        print(f"✅ Loading real data from: {path}")
        df = pd.read_csv(path)
        return df
    
    print("⚠️ No CSV found, generating synthetic demo data instead...")
    rng = np.random.default_rng(random_state)
    
    n = n_rows
    df = pd.DataFrame({
        "transaction_id": np.arange(1, n + 1),
        "customer_id": rng.integers(1, 1000, size=n),
        "amount": np.round(rng.gamma(shape=2.0, scale=40.0, size=n), 2),
        "merchant_category": rng.choice(
            ["grocery", "electronics", "travel", "gaming", "restaurants", "gas"], 
            size=n, 
            replace=True
        ),
        "channel": rng.choice(["ecom", "pos", "ivr", "mobile"], size=n, replace=True),
        "country": rng.choice(["US", "CA", "GB", "AU", "DE"], size=n, replace=True),
    })
    
    # Create a synthetic fraud pattern
    base_prob = 0.03
    prob = np.full(n, base_prob)
    
    # Higher risk patterns
    prob += np.where(df["amount"] > 250, 0.04, 0.0)
    prob += np.where(df["merchant_category"].isin(["electronics", "travel", "gaming"]), 0.03, 0.0)
    prob += np.where(df["channel"].isin(["ecom", "mobile"]), 0.02, 0.0)
    prob += np.where(df["country"].isin(["GB", "DE"]), 0.01, 0.0)
    
    prob = np.clip(prob, 0, 0.95)
    df["is_fraud"] = rng.binomial(1, prob)
    
    print("✅ Synthetic dataset created with shape:", df.shape)
    print("Fraud rate:", df["is_fraud"].mean())
    return df

df = load_or_generate_data(DATA_PATH)
df.head()


## 2. Quick EDA

In [ ]:

print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())

print("\nClass balance (is_fraud):")
print(df["is_fraud"].value_counts(normalize=True).rename("proportion"))

print("\nAmount stats:")
print(df["amount"].describe())

print("\nMerchant category counts:")
print(df["merchant_category"].value_counts().head())


## 3. Feature engineering & preprocessing

In [ ]:

# Define features and target
TARGET_COL = "is_fraud"

# Keep a simple feature set for now; customize for real project
numeric_features = ["amount"]
categorical_features = ["merchant_category", "channel", "country"]

feature_cols = numeric_features + categorical_features

df_model = df[feature_cols + [TARGET_COL]].dropna()

X = df_model[feature_cols]
y = df_model[TARGET_COL]

print("Modeling dataset shape:", X.shape)


In [ ]:

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


In [ ]:

# Preprocess numeric + categorical features
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


## 4. Train models

In [ ]:

# Logistic Regression pipeline
log_reg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

# Random Forest pipeline
rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        n_jobs=-1,
        class_weight="balanced_subsample",
        random_state=42
    ))
])

models = {
    "log_reg": log_reg_pipeline,
    "random_forest": rf_pipeline
}

fitted_models = {}

for name, model in models.items():
    print(f"\n🚀 Training model: {name}")
    model.fit(X_train, y_train)
    fitted_models[name] = model
    
    # Evaluate quickly on test set
    y_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    print(f"AUC ({name}): {auc:.4f}")


## 5. Detailed evaluation for champion model

In [ ]:

# Choose champion model here
champion_name = "random_forest"  # change to "log_reg" if you prefer
champion_model = fitted_models[champion_name]

print(f"Using champion model: {champion_name}")

y_proba = champion_model.predict_proba(X_test)[:, 1]
y_pred_default = (y_proba >= 0.5).astype(int)

print("\nAUC:", roc_auc_score(y_test, y_proba))
print("\nClassification report (threshold=0.5):")
print(classification_report(y_test, y_pred_default))

print("\nConfusion matrix (threshold=0.5):")
print(confusion_matrix(y_test, y_pred_default))


## 6. Threshold tuning

In [ ]:

precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

# Example strategy: pick threshold that gives at least target_recall
target_recall = 0.80
best_threshold = 0.5
best_precision = 0
best_recall = 0

for p, r, t in zip(precision, recall, np.append(thresholds, 1.0)):
    if r >= target_recall and p > best_precision:
        best_precision = p
        best_recall = r
        best_threshold = t

print(f"Chosen threshold: {best_threshold:.4f}")
print(f"Precision at chosen threshold: {best_precision:.4f}")
print(f"Recall at chosen threshold: {best_recall:.4f}")

# Evaluate at chosen threshold
y_pred_tuned = (y_proba >= best_threshold).astype(int)
print("\nClassification report (tuned threshold):")
print(classification_report(y_test, y_pred_tuned))

print("\nConfusion matrix (tuned threshold):")
print(confusion_matrix(y_test, y_pred_tuned))


## 7. Scoring function for new transactions

In [ ]:

def score_transactions(
    df_new: pd.DataFrame,
    model: Pipeline,
    threshold: float,
    id_col: str = "transaction_id"
) -> pd.DataFrame:
    """Score new transactions and flag those above a risk threshold.
    
    Parameters
    ----------
    df_new : pd.DataFrame
        New transactions. Must contain the same feature columns used in training.
    model : Pipeline
        Fitted sklearn Pipeline with a `predict_proba` method.
    threshold : float
        Risk score threshold above which to flag transactions.
    id_col : str
        Column name to treat as primary key / transaction identifier.
    """
    missing_cols = [c for c in feature_cols if c not in df_new.columns]
    if missing_cols:
        raise ValueError(f"df_new is missing required columns: {missing_cols}")
    
    X_new = df_new[feature_cols]
    scores = model.predict_proba(X_new)[:, 1]
    flags = (scores >= threshold).astype(int)
    
    out = df_new.copy()
    out["risk_score"] = scores
    out["is_flagged"] = flags
    return out[[id_col] + feature_cols + ["risk_score", "is_flagged"]] if id_col in out.columns else out

# Demo with a few rows from the test set
demo = X_test.copy().head(5)
demo["transaction_id"] = np.arange(1, len(demo) + 1)
scored_demo = score_transactions(demo, champion_model, best_threshold)
scored_demo


## 8. Persist champion model

In [ ]:

MODEL_PATH = "models/ai_transaction_guardian_champion.joblib"
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

joblib.dump({
    "model": champion_model,
    "feature_cols": feature_cols,
    "numeric_features": numeric_features,
    "categorical_features": categorical_features,
    "threshold": best_threshold,
    "champion_name": champion_name,
}, MODEL_PATH)

print(f"✅ Champion model artifact saved to: {MODEL_PATH}")
